In [1]:
"""Demo for use yolo v3
"""
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO


In [2]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
# 10->traffic light
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
        
        print(f"cl :{cl}")
        if cl == 9:
            cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
            cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                        (top, left - 6),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 0, 255), 1,
                        cv2.LINE_AA)
#             for (root, dirs, files) in os.walk('images/test'):
#             if files:
#                 for f in files:
#                     print(f)
#                     path = os.path.join(root, f)
#                     image = cv2.imread(path)
#                     image = detect_image(image, yolo, all_classes)
#                     cv2.imwrite('images/res/' + f, image)
            crop_area = image[left:bottom,top:right]
            cv2.imwrite('images/crop_area/' + f, crop_area)
#            cv2.imwrite('C:/Users/abhir/YOLOv3-master/images/croped_img',crop_area)
    
            

#         cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
#         cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
#                     (top, left - 6),
#                     cv2.FONT_HERSHEY_SIMPLEX,
#                     0.6, (0, 0, 255), 1,
#                     cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [5]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [6]:

def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)
    
#     cv2.namedWindow("detection", cv2.WINDOW_NORMAL)
#     cv2.resizeWindow("detection", 900, 900) 

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(30) & 0xff == 'q':
                break

    vout.release()
    camera.release()

In [11]:

# if __name__ == '__main__':
#     yolo = YOLO(0.6, 0.5)
#     file = 'data/coco_classes.txt'
#     all_classes = get_classes(file)


yolo = YOLO(0.6, 0.3)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

In [12]:

    # detect images in test floder.
    for (root, dirs, files) in os.walk('images/test'):
        if files:
            for f in files:
                print(f)
                path = os.path.join(root, f)
                image = cv2.imread(path)
                image = detect_image(image, yolo, all_classes)
                cv2.imwrite('images/res/' + f, image)

    # detect videos one at a time in videos/test folder    
#     video = 'library1.mp4'
#     detect_video(video, yolo, all_classes)


dog.jpg
time: 4.82s
cl :16
class: dog, score: 1.00
box coordinate x,y,w,h: [124.10609436 220.43732643 195.35037231 321.95930804]
cl :1
class: bicycle, score: 1.00
box coordinate x,y,w,h: [119.10217667 118.63843918 448.7917099  321.94878453]
cl :7
class: truck, score: 0.91
box coordinate x,y,w,h: [468.68074036  84.48197079 227.29340744  83.52550397]

giraffe.jpg
time: 2.36s
cl :22
class: zebra, score: 0.97
box coordinate x,y,w,h: [271.4728713  197.00688124 157.91193768 260.55246592]
cl :23
class: giraffe, score: 1.00
box coordinate x,y,w,h: [156.34179115  38.96565735 288.86812316 359.70112567]

t1.jpg
time: 2.36s
cl :9
class: traffic light, score: 1.00
box coordinate x,y,w,h: [118.63767475  23.78337979 185.92112795 374.70880079]
cl :9
class: traffic light, score: 0.76
box coordinate x,y,w,h: [420.76882267 142.55504751  73.44887322 113.8915897 ]

t2.jpg
time: 2.72s
cl :9
class: traffic light, score: 0.98
box coordinate x,y,w,h: [175.20989227  40.70384018  54.58356714 120.3732653 ]

t3.jp

In [ ]:
    video = 'T1.mp4'
    detect_video(video, yolo, all_classes)